In [3]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
def myxlogx(x):
    if x>0:
        return (x*np.log2(x))
    else:
        return 0
    
#calculating entropy of feature X i.e. H(X)   
def entropy(x):
    counter_values = Counter(x).most_common()
    probabilities = [elem[1]/len(x) for elem in counter_values]
    entropy=-sum(map(myxlogx,probabilities))
    return entropy

#calculating entropy of pair of features X1,X2 i.e. H(X1,X2)
def entropy2(x1,x2):
    x=[tuple(elem) for elem in zip(x1,x2)]
    counter_values=Counter(x).most_common()
    probabilities = [elem[1]/len(x) for elem in counter_values]
    #entropy=scipy.stats.entropy(probabilities)
    entropy=-sum(map(myxlogx,probabilities))
    return entropy

#calculating entropy of 3-pair of features X1,X2,X3 i.e. H(X1,X2,X3)
def entropy3(x1,x2,x3):
    x=[tuple(elem) for elem in zip(x1,x2,x3)]
    counter_values=Counter(x).most_common()
    probabilities = [elem[1]/len(x) for elem in counter_values]
    #entropy=scipy.stats.entropy(probabilities)
    entropy=-sum(map(myxlogx,probabilities))
    return entropy

#Calculating mutual information of features I(x;y)
def MI(x,y):
    return (entropy(x)+entropy(y)-entropy2(x,y))

#Calculate mutual information of features I(x1,x2;y)
def MI2(x1,x2,y):
    return (entropy2(x1,x2)+entropy(y)-entropy3(x1,x2,y))

#Calculate conditional mutual information of features I(x1;y|x2)
def CMI2(x1,x2,y):
    return (-entropy3(x1,x2,y)-entropy(y)+entropy2(x1,y)+entropy2(x2,y))

#Calculate joint mutual information of features I(x1,x2;y)
def jointMI(x1,x2,y):
    return CMI2(x1,y,x2)+MI(x2,y)

def CMIM(X,y,K=10):
    N=X.shape[1]
    ps=np.zeros(N)
    nu=np.zeros(N).astype(np.int)
    
    for n in range(N):
        ps[n]=MI(X[:,n],y)

    for k in range(N):
        nu[k]=np.where(ps==max(ps))[0][0]
        for n in range(N):
            ps[n]=min(ps[n],CMI2(X[:,n],y,X[:,nu[k]]))
        if(k==K):
            break
    return ps,nu

def JMI(X,y):
    N=X.shape[1]
    nu=np.zeros(N).astype(np.int)
    mi_score=np.zeros(N)
    for n in range(N):
        mi_score[n]=MI(X[:,n],y)

    for k in range(N):
        j_score=0
        nu[k]=np.where(mi_score==max(mi_score))[0][0]
        for i in range(N):
            if(i!=nu[k]):
                j_score+=MI2(X[:,nu[k]],X[:,i],y)
        mi_score[nu[k]]=j_score
    return mi_score

def DISR(X,y):
    N=X.shape[1]
    nu=np.zeros(N).astype(np.int)
    mi_score=np.zeros(N)
    for n in range(N):
        mi_score[n]=MI(y,X[:,n])

    for k in range(N):
        j_score=0
        nu[k]=np.where(mi_score==max(mi_score))[0][0]
        for i in range(N):
            if(i!=nu[k]):
                j_score+=((MI2(X[:,nu[k]],X[:,i],y))/entropy3(X[:,nu[k]],X[:,i],y))
        mi_score[nu[k]]=j_score
    return mi_score